In [1]:
!python -m pip install --upgrade pip setuptools wheel

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 7.7 MB/s  0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 72.1.0
    Uninstalling setuptools-72.1.0:
      Successfully uninstalled setuptools-72.1.0


In [2]:
!pip install --use-pep517 seqeval transformers datasets sklearn-crfsuite accelerate tokenizers

In [4]:
# Paste and run this whole cell
import json, random, os
from pathlib import Path
from pprint import pprint

random.seed(42)
EDA_DIR = Path("finer139_eda_outputs")
SRC = EDA_DIR / "train_sample_500.jsonl"
OUT_DIR = Path("data")
OUT_DIR.mkdir(exist_ok=True)

if not SRC.exists():
    raise FileNotFoundError(f"{SRC} not found. Check folder and filename.")

# 1) Inspect first N examples to discover keys
print("Inspecting first 20 lines of", SRC)
sample_objs = []
with open(SRC, encoding="utf-8") as f:
    for i, line in enumerate(f):
        if not line.strip(): 
            continue
        try:
            obj = json.loads(line)
        except Exception as e:
            print("Error parsing line", i, e)
            continue
        sample_objs.append(obj)
        if i >= 19:
            break

# show keys distribution in sample
from collections import Counter, defaultdict
key_counts = Counter()
example_keys = []
for i,obj in enumerate(sample_objs):
    ks = tuple(sorted(obj.keys()))
    key_counts.update(ks)
    example_keys.append(ks)

print("Unique key-sets seen in first 20 lines (counts):")
pprint(key_counts.most_common())

print("\nExample key-sets for first 5 examples:")
for i,ks in enumerate(example_keys[:5]):
    print(i, ks)

print("\nShowing raw structure of the first 3 examples (truncated):")
for i,obj in enumerate(sample_objs[:3]):
    print(f"\n--- Example {i} keys: {list(obj.keys())} ---")
    # print small preview of each key value
    for k in obj.keys():
        v = obj[k]
        # pretty print short values, avoid huge prints
        if isinstance(v, list):
            print(f"{k} -> list(len={len(v)}) sample:", v[:10])
        elif isinstance(v, dict):
            print(f"{k} -> dict keys:", list(v.keys())[:10])
        else:
            print(f"{k} -> {str(v)[:200]}")
    # stop if structure shows tokens/tags
    if i >= 2:
        pass

# 2) Try to automatically convert all examples into {"id","tokens","tags"} format
# Heuristics supported:
# - tokens can be under "tokens", "words", "tokens_list", "text_tokens"
# - tags can be under "tags", "ner_tags", "labels", "label", "bio_tags"
# - if ner_tags are integers we will convert them to string labels like "LABEL_{int}"
# - If tokens exist but no tag-like field exists, we will assign "O" for all tokens
# - If tokens missing but "text" exists, we cannot safely tokenize to BIO automatically (we'll print a warning).

def detect_tokens_field(obj):
    for candidate in ("tokens","words","tokens_list","text_tokens","tokenized","tokens_final"):
        if candidate in obj:
            return candidate
    # sometimes tokens can be under "tokens" but nested; fallback look
    for k in obj:
        if isinstance(obj[k], list) and len(obj[k])>0 and isinstance(obj[k][0], str) and len(obj[k][0].split())<=3:
            # likely tokens
            return k
    return None

def detect_tags_field(obj):
    for candidate in ("tags","ner_tags","labels","bio_tags","entity_tags","ner_labels"):
        if candidate in obj:
            return candidate
    # sometimes tags are under 'labels' as ints or strings
    for k in obj:
        if isinstance(obj[k], list) and len(obj[k])>0:
            # tags are usually small strings like 'B-ORG' or ints
            if all(isinstance(x, (str,int)) for x in obj[k]):
                # element lengths small
                return k
    return None

def normalize_example(obj, idx):
    # returns (normalized_obj, error_message_or_None)
    tokens_key = detect_tokens_field(obj)
    tags_key = detect_tags_field(obj)
    norm = {}
    # id
    norm['id'] = obj.get('id', f"line_{idx}")
    # tokens
    if tokens_key:
        tokens = obj[tokens_key]
        if not isinstance(tokens, list):
            # if it's a string, try splitting by whitespace
            if isinstance(tokens, str):
                tokens = tokens.split()
            else:
                return None, f"tokens field {tokens_key} not list or str (type {type(tokens)})"
    elif 'text' in obj and isinstance(obj['text'], str):
        # fallback: if only plain text present, we can't convert reliably. Return error to manual fix.
        return None, "No token list found, only 'text' present. Manual tokenization needed."
    else:
        return None, "No token-like field found"
    # tags
    if tags_key:
        tags = obj[tags_key]
        # if tags are ints -> convert to LABEL_{i}
        if all(isinstance(x, int) for x in tags):
            tags = [f"LABEL_{x}" for x in tags]
        elif all(isinstance(x, str) for x in tags):
            # ok
            pass
        else:
            # mixed or unknown
            tags = [str(x) for x in tags]
    else:
        # no tag field found: assign "O" for all tokens (safe fallback)
        tags = ["O"] * len(tokens)

    if len(tokens) != len(tags):
        return None, f"Length mismatch after normalization: tokens {len(tokens)} vs tags {len(tags)} (id={norm['id']})"

    norm['tokens'] = tokens
    norm['tags'] = tags
    return norm, None

# Now convert all examples in the source file
converted = []
errors = []
with open(SRC, encoding="utf-8") as f:
    for i, line in enumerate(f):
        if not line.strip(): continue
        try:
            obj = json.loads(line)
        except Exception as e:
            errors.append((i, f"JSON parse error: {e}"))
            continue
        norm, err = normalize_example(obj, i)
        if err:
            errors.append((i, err, obj if i<5 else None))  # include obj for first few errors
        else:
            converted.append(norm)

print(f"\nConversion complete. Converted {len(converted)} examples, {len(errors)} errors.")
if errors:
    print("First 10 errors (index, message):")
    for e in errors[:10]:
        print(e[0], "->", e[1])
        if len(e) > 2 and e[2] is not None:
            print("Sample offending object keys:", list(e[2].keys()))
            # print small preview
            pprint({k: (e[2][k][:10] if isinstance(e[2][k], list) else str(e[2][k])[:200]) for k in list(e[2].keys())[:8]})
            print("---")

# If many errors occurred, stop and print instructions
if len(errors) > 0:
    print("\nThere are errors that the automatic converter couldn't fix.")
    print("Common causes: examples that only contain full 'text' (not tokens), or entity spans (start/end) instead of per-token tags.")
    print("If errors are few, we can inspect and fix them manually. If many, re-run EDA conversion to produce 'tokens' + 'tags' lists.")
else:
    # write split files to data/ as dataset_train.jsonl and dataset_valid.jsonl (90/10 split)
    random.shuffle(converted)
    n = int(0.9 * len(converted))
    train_conv = converted[:n]
    valid_conv = converted[n:]
    train_out = OUT_DIR / "dataset_train.jsonl"
    valid_out = OUT_DIR / "dataset_valid.jsonl"
    with open(train_out, "w", encoding="utf-8") as fw:
        for ex in train_conv:
            fw.write(json.dumps(ex, ensure_ascii=False) + "\n")
    with open(valid_out, "w", encoding="utf-8") as fw:
        for ex in valid_conv:
            fw.write(json.dumps(ex, ensure_ascii=False) + "\n")
    print(f"Wrote {len(train_conv)} converted train and {len(valid_conv)} converted valid examples to {OUT_DIR}/")

    # Re-validate lengths
    bad = 0
    for name,p in [("train", train_out), ("valid", valid_out)]:
        for i,line in enumerate(open(p, encoding="utf-8")):
            obj = json.loads(line)
            if len(obj["tokens"]) != len(obj["tags"]):
                print("MISMATCH", name, "line", i, "id", obj.get("id"), "tokens", len(obj["tokens"]), "tags", len(obj["tags"]))
                bad += 1
    print("Total mismatches after conversion:", bad)
    if bad == 0:
        print("✅ All good — conversion produced token/tag aligned JSONL. You can now continue with tokenization & training.")


Inspecting first 20 lines of finer139_eda_outputs\train_sample_500.jsonl
Unique key-sets seen in first 20 lines (counts):
[('id', 20),
 ('ner_tags', 20),
 ('token_count', 20),
 ('tokens', 20),
 ('tokens_norm', 20)]

Example key-sets for first 5 examples:
0 ('id', 'ner_tags', 'token_count', 'tokens', 'tokens_norm')
1 ('id', 'ner_tags', 'token_count', 'tokens', 'tokens_norm')
2 ('id', 'ner_tags', 'token_count', 'tokens', 'tokens_norm')
3 ('id', 'ner_tags', 'token_count', 'tokens', 'tokens_norm')
4 ('id', 'ner_tags', 'token_count', 'tokens', 'tokens_norm')

Showing raw structure of the first 3 examples (truncated):

--- Example 0 keys: ['id', 'tokens', 'ner_tags', 'token_count', 'tokens_norm'] ---
id -> 0
tokens -> list(len=327) sample: ['ITEM', '1', 'Financial', 'Statements', 'Lennar', 'Corporation', 'and', 'Subsidiaries', 'Condensed', 'Consolidated']
ner_tags -> list(len=327) sample: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
token_count -> 327
tokens_norm -> list(len=327) sampl

In [5]:
# Cell: imports & device
import json, os
from pathlib import Path
import random, numpy as np
random.seed(42); np.random.seed(42)

import torch
print("torch device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

from sklearn_crfsuite import CRF
from seqeval.metrics import classification_report, precision_score, recall_score, f1_score

from datasets import Dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, pipeline


torch device: cpu



In [6]:
# Cell: load data and build labels
train_raw = [json.loads(l) for l in open("data/dataset_train.jsonl", encoding="utf-8")]
valid_raw = [json.loads(l) for l in open("data/dataset_valid.jsonl", encoding="utf-8")]
print("Train examples:", len(train_raw), "Valid examples:", len(valid_raw))
print("Sample tokens:", train_raw[0]["tokens"][:20])
print("Sample tags  :", train_raw[0]["tags"][:20])

# build label list (deterministic)
label_list = sorted({ tag for ex in train_raw for tag in ex["tags"] })
label2id = {l:i for i,l in enumerate(label_list)}
id2label = {i:l for l,i in label2id.items()}
print("Num labels:", len(label_list))
print("First 40 labels:", label_list[:40])


Train examples: 450 Valid examples: 50
Sample tokens: ['A', 'subsidiary', "'s", 'guarantee', 'will', 'be', 'suspended', 'at', 'any', 'time', 'when', 'it', 'is', 'not', 'directly', 'or', 'indirectly', 'guaranteeing', 'at', 'least']
Sample tags  : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Num labels: 39
First 40 labels: ['B-AllocatedShareBasedCompensationExpense', 'B-AmortizationOfIntangibleAssets', 'B-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount', 'B-AreaOfRealEstateProperty', 'B-BusinessAcquisitionPercentageOfVotingInterestsAcquired', 'B-ClassOfWarrantOrRightExercisePriceOfWarrantsOrRights1', 'B-DebtInstrumentBasisSpreadOnVariableRate1', 'B-DebtInstrumentCarryingAmount', 'B-DebtInstrumentConvertibleConversionPrice1', 'B-DebtInstrumentFaceAmount', 'B-DebtInstrumentFairValue', 'B-DebtInstrumentInterestRateEffectivePercentage', 'B-DebtInstrumentInterestRateStatedPercentage', 'B-DebtInstrumentRedemptionPric

In [7]:
# Cell: CRF baseline (interpretable baseline)
def token2features(sent, i):
    token = sent[i]
    feats = {
        "bias":1.0,
        "word.lower()": token.lower(),
        "word[-3:]": token[-3:],
        "word.isdigit()": token.isdigit(),
        "word.istitle()": token.istitle()
    }
    if i>0:
        t=sent[i-1]; feats["-1:word.lower()"]=t.lower()
    else:
        feats["BOS"]=True
    if i<len(sent)-1:
        t=sent[i+1]; feats["+1:word.lower()"]=t.lower()
    else:
        feats["EOS"]=True
    return feats

def sent2features(sent): return [token2features(sent,i) for i in range(len(sent))]

X_train = [sent2features(ex["tokens"]) for ex in train_raw]
y_train = [ex["tags"] for ex in train_raw]
X_val = [sent2features(ex["tokens"]) for ex in valid_raw]
y_val = [ex["tags"] for ex in valid_raw]

print("Training CRF baseline (this is quick)...")
crf = CRF(algorithm="lbfgs", max_iterations=200)
crf.fit(X_train, y_train)
y_pred = crf.predict(X_val)

crf_report = classification_report(y_val, y_pred)
print("CRF entity-level classification report:\n")
print(crf_report)

# save small CRF report file
with open("results_crf_report.txt","w",encoding="utf-8") as f:
    f.write(crf_report)
print("Saved results_crf_report.txt")


Training CRF baseline (this is quick)...
CRF entity-level classification report:

                                             precision    recall  f1-score   support

     AllocatedShareBasedCompensationExpense       0.00      0.00      0.00         4
 DebtInstrumentInterestRateStatedPercentage       1.00      1.00      1.00         1
                    DeferredFinanceCostsNet       0.00      0.00      0.00         2
  EquityMethodInvestmentOwnershipPercentage       0.00      0.00      0.00         1
      IncomeLossFromEquityMethodInvestments       0.00      0.00      0.00         4
           LettersOfCreditOutstandingAmount       0.00      0.00      0.00         2
                               LongTermDebt       0.00      0.00      0.00         2
          LossContingencyDamagesSoughtValue       0.00      0.00      0.00         1
MinorityInterestOwnershipPercentageByParent       0.00      0.00      0.00         2

                                  micro avg       1.00      0.05  

C:\Users\lokes\anaconda3\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
# run this cell to confirm current working dir and files
import os
from pathlib import Path

print("Working directory:", Path.cwd())
print("\nTop-level files/folders:")
for p in Path(".").iterdir():
    print("-", p.name)

print("\nContents of finer139_eda_outputs (if present):")
eda = Path("finer139_eda_outputs")
if eda.exists():
    for p in sorted(eda.iterdir()):
        print("  -", p.name)
else:
    print("  finer139_eda_outputs folder NOT found in the current directory.")


Working directory: C:\Users\lokes

Top-level files/folders:
- .anaconda
- .azuredatastudio
- .cache
- .conda
- .condarc
- .continuum
- .gitconfig
- .idlerc
- .ipynb_checkpoints
- .ipython
- .jupyter
- .keras
- .matplotlib
- .ms-ad
- .vscode
- add.py
- anaconda3
- AppData
- Application Data
- atag.html
- battery-report.html
- bytes.py
- civic-issue-app
- Contacts
- Cookies
- data
- data typess.py
- dataset
- date and time.py
- directorys.py
- DL programs.ipynb
- Documents
- eda output.ipynb
- Favorites
- finenv
- finer139
- finer139.zip
- finer139_cleaned.csv
- finer139_eda_outputs
- fire_events
- functions.py
- get-pip.py
- if,else.py
- Infosys springboard(EDA) output.ipynb
- Infosys_springboard.ipynb
- java
- Links
- ll.py
- Local Settings
- looping.py
- Microsoft
- ML challenge.ipynb
- Music
- My Documents
- nested function with argument.py
- NetHood
- New folder
- NTUSER.DAT
- ntuser.dat.LOG1
- ntuser.dat.LOG2
- NTUSER.DAT{2ad838bc-efea-11ee-a54d-000d3a94eaa1}.TM.blf
- NTUSER.DAT{2a

In [1]:
# ✅ Cell 1: Create clean data files for training
import json, random
from pathlib import Path

random.seed(42)
EDA_SRC = Path("finer139_eda_outputs") / "train_sample_500.jsonl"
OUT_DIR = Path("data")
OUT_DIR.mkdir(exist_ok=True)

# Load and normalize
examples = []
with open(EDA_SRC, encoding="utf-8") as f:
    for i,line in enumerate(f):
        obj = json.loads(line)
        tokens = [str(t) for t in obj.get("tokens",[])]
        tags = obj.get("tags") or obj.get("ner_tags") or obj.get("labels") or []
        tags = [str(t).strip() for t in tags]
        if len(tokens) == len(tags):
            examples.append({"id": obj.get("id", i), "tokens": tokens, "tags": tags})

print("Loaded examples:", len(examples))

random.shuffle(examples)
n = int(len(examples) * 0.9)
train = examples[:n]
valid = examples[n:]

# Save
with open("data/dataset_train.jsonl", "w", encoding="utf-8") as f:
    for ex in train: f.write(json.dumps(ex)+"\n")

with open("data/dataset_valid.jsonl", "w", encoding="utf-8") as f:
    for ex in valid: f.write(json.dumps(ex)+"\n")

print("✅ Saved cleaned train (", len(train), ") and valid (", len(valid), ")")


Loaded examples: 500
✅ Saved cleaned train ( 450 ) and valid ( 50 )


In [2]:
# ✅ Cell 2: Validate cleaned data
import json

bad = 0
for name, path in (("train", "data/dataset_train.jsonl"), ("valid", "data/dataset_valid.jsonl")):
    for i, line in enumerate(open(path, encoding="utf-8")):
        obj = json.loads(line)
        if len(obj["tokens"]) != len(obj["tags"]):
            print("Mismatch:", name, i)
            bad += 1

print("✅ Validation complete — mismatches:", bad)


✅ Validation complete — mismatches: 0


In [5]:
# ✅ FINAL Cell 3 — Tokenize & align labels with ALL dataset tags
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

MODEL_START = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_START, use_fast=True)

train_data = [json.loads(l) for l in open('data/dataset_train.jsonl', encoding='utf-8')]
valid_data = [json.loads(l) for l in open('data/dataset_valid.jsonl', encoding='utf-8')]
dataset = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(valid_data)
})

# ✅ Build label list including ALL labels from train + valid
label_list = sorted({
    tag for split in dataset.values()
    for ex in split
    for tag in ex["tags"]
})
label2id = {l:i for i,l in enumerate(label_list)}
id2label = {i:l for l,i in label2id.items()}

max_length = 128

def tokenize_and_align_labels(batch):
    tokenized = tokenizer(batch["tokens"], is_split_into_words=True,
                          truncation=True, padding="max_length",
                          max_length=max_length)
    aligned = []
    for i, labs in enumerate(batch["tags"]):
        ids = tokenized.word_ids(batch_index=i)
        prev, row = None, []
        for wid in ids:
            if wid is None:
                row.append(-100)
            elif wid != prev:
                row.append(label2id[labs[wid]])
            else:
                row.append(-100)
            prev = wid
        aligned.append(row)
    tokenized["labels"] = aligned
    return tokenized

print("⏳ Tokenizing with ALL labels — 139+ tags…")
tokenized = dataset.map(tokenize_and_align_labels, batched=True,
                        remove_columns=dataset["train"].column_names)
tokenized.save_to_disk("tokenized_dataset")
print("✅ Tokenized dataset saved → tokenized_dataset/")
print("✅ Total labels:", len(label_list))


⏳ Tokenizing with ALL labels — 139+ tags…


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/450 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

✅ Tokenized dataset saved → tokenized_dataset/
✅ Total labels: 42


In [7]:
# ✅ Fixed Cell 4: Train model (compatible with your version)
import evaluate
import numpy as np
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

metric = evaluate.load("seqeval")

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_START,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=-1)
    labels = p.label_ids
    pred_tags, true_tags = [], []
    for p_row, l_row in zip(preds, labels):
        for p_val, l_val in zip(p_row, l_row):
            if l_val != -100:
                pred_tags.append(id2label[p_val])
                true_tags.append(id2label[l_val])

    results = metric.compute(predictions=[pred_tags], references=[true_tags])
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

training_args = TrainingArguments(
    output_dir="./model_finer139",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=50,        # ✅ replaces evaluation_strategy
    eval_steps=200,          # ✅ run eval occasionally
    save_steps=500
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()
eval_results = trainer.evaluate()
print("\n📊 Final Evaluation Results:", eval_results)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\lokes\AppData\Local\Temp\ipykernel_17052\2779100780.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
C:\Users\lokes\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,0.319600
100,0.050400
150,0.049900
200,0.046000


C:\Users\lokes\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



📊 Final Evaluation Results: {'eval_loss': 0.05491551384329796, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.9902650081124933, 'eval_runtime': 32.2417, 'eval_samples_per_second': 1.551, 'eval_steps_per_second': 0.775, 'epoch': 1.0}


C:\Users\lokes\anaconda3\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lokes\anaconda3\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
